<a href="https://colab.research.google.com/github/juliendymendes/pet-classifier/blob/main/trabalhoia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import urllib.request
import os
from PIL import Image,ImageStat
import numpy as np
import matplotlib.pyplot as plt

import torchvision
import torch
import torch.nn as nn
from torch.utils.data import  DataLoader
import torchvision.transforms as transforms
from torchvision import transforms, datasets

from torch import optim
import sklearn.metrics as metrics

In [2]:
import torch.nn.functional as F

In [3]:
main_dir = "/content/drive/MyDrive/animais"

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
os.chdir(main_dir)

In [6]:
# Define transformations to apply to the images
transform = transforms.Compose([
    transforms.Resize((256, 256)),  # Resize images to 256x256
    transforms.CenterCrop(224),     # Crop the center 224x224 region
    transforms.ToTensor(),           # Convert images to tensors
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize the images
])

# Create an ImageFolder dataset
image_dataset = datasets.ImageFolder(root=main_dir, transform=transform)

In [7]:
image_dataset.class_to_idx

{'cachorro': 0, 'gato': 1}

In [8]:
def image_loader(img):
    return Image.open(img).convert("RGB")

In [9]:
model = torchvision.models.regnet_y_400mf()

In [10]:
model.fc = nn.Linear(440,2)

In [11]:
model

RegNet(
  (stem): SimpleStemIN(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (trunk_output): Sequential(
    (block1): AnyStage(
      (block1-0): ResBottleneckBlock(
        (proj): Conv2dNormActivation(
          (0): Conv2d(32, 48, kernel_size=(1, 1), stride=(2, 2), bias=False)
          (1): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (f): BottleneckTransform(
          (a): Conv2dNormActivation(
            (0): Conv2d(32, 48, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (1): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU(inplace=True)
          )
          (b): Conv2dNormActivation(
            (0): Conv2d(48, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=6, bias=False)
            

In [12]:
dataset = image_dataset

In [13]:
len(dataset)


400

In [14]:
dataset = torchvision.datasets.DatasetFolder('./',loader = image_loader,extensions='jpg',transform=transform )

In [15]:
dataset.targets = torch.tensor(dataset.targets)

In [16]:
ds = image_dataset

In [ ]:
ds

In [17]:
from sklearn.model_selection import train_test_split

In [18]:
np.unique(dataset.targets,return_counts=True)

(array([0, 1]), array([190, 177]))

In [19]:
ds.targets = np.array(ds.targets)

In [20]:
bs = 64
train_idx, temp_idx = train_test_split(np.arange(len(ds)),test_size=0.3,shuffle=True,stratify=ds.targets)
valid_idx, test_idx = train_test_split(temp_idx,test_size=0.5,shuffle=True,stratify=ds.targets[temp_idx])

train_sampler = torch.utils.data.SubsetRandomSampler(train_idx)
valid_sampler = torch.utils.data.SubsetRandomSampler(valid_idx)
test_sampler  = torch.utils.data.SubsetRandomSampler(test_idx)

dl_train = torch.utils.data.DataLoader(ds,batch_size=bs,sampler=train_sampler)
dl_valid = torch.utils.data.DataLoader(ds,batch_size=bs,sampler=valid_sampler)
dl_test  = torch.utils.data.DataLoader(ds,batch_size=bs,sampler=test_sampler)

In [21]:
x,y = next(iter(dl_train))

In [22]:
device = torch.device('cuda:0')

In [ ]:
x.to(device)

In [ ]:
model.to(device)

In [24]:
loss_train = []
loss_eval  = []
patience_time = 15

In [25]:
criterion = nn.CrossEntropyLoss()
opt = optim.SGD(model.parameters(),lr=0.01)

In [28]:
epochs = 100

stop = False
epoch = 0
lowest_loss_eval = 10000
last_best_result = 0
while (not stop):
    #wandb.watch(model)
    model.train()
    lloss = []
    for x,y in dl_train:
        x = x.to(device)
        y = y.to(device)
        pred = model(x)
        closs = criterion(pred,y)
        closs.backward()
        opt.step()
        opt.zero_grad()
        lloss.append(closs.item())
        #print(closs.item())
    loss_train.append(np.mean(lloss))
    lloss = []
    model.eval()
    lres = []
    ytrue = []
    with torch.no_grad():
        for data,y in dl_valid:
            data = data.to(device)

            pred = model(data)
            closs = criterion(pred.cpu(),y)
            lloss.append(closs.item())
            res  = pred.argmax(dim=1).cpu().tolist()
            lres += res
            ytrue += y
    avg_loss_eval = np.mean(lloss)
    loss_eval.append(avg_loss_eval)
    #wandb.log({"loss_eval": avg_loss_eval,"loss_train":loss_train[-1]})
    if avg_loss_eval < lowest_loss_eval:
        lowest_loss_eval = avg_loss_eval
        last_best_result = 0
        print("Best model found! saving...")
        actual_state = {'optim':opt.state_dict(),'model':model.state_dict(),'epoch':epoch,'loss_train':loss_train,'loss_eval':loss_eval}
        torch.save(actual_state,'best_model.pth')
    last_best_result += 1
    if last_best_result > patience_time:
        stop = True
    print("epoch %d loss_train %4.3f loss_eval %4.3f last_best %d"%(epoch,loss_train[-1],loss_eval[-1],last_best_result))
    epoch += 1

Best model found! saving...
epoch 0 loss_train 1.009 loss_eval 0.740 last_best 1
epoch 1 loss_train 1.237 loss_eval 0.902 last_best 2
epoch 2 loss_train 1.513 loss_eval 0.994 last_best 3
epoch 3 loss_train 1.084 loss_eval 0.801 last_best 4
epoch 4 loss_train 1.093 loss_eval 1.244 last_best 5
epoch 5 loss_train 1.439 loss_eval 0.781 last_best 6
epoch 6 loss_train 0.747 loss_eval 0.976 last_best 7
Best model found! saving...
epoch 7 loss_train 0.787 loss_eval 0.713 last_best 1
Best model found! saving...
epoch 8 loss_train 0.732 loss_eval 0.695 last_best 1
epoch 9 loss_train 0.727 loss_eval 0.917 last_best 2
Best model found! saving...
epoch 10 loss_train 0.733 loss_eval 0.692 last_best 1
epoch 11 loss_train 0.650 loss_eval 0.863 last_best 2
epoch 12 loss_train 0.709 loss_eval 2.291 last_best 3
epoch 13 loss_train 0.839 loss_eval 0.885 last_best 4
epoch 14 loss_train 0.695 loss_eval 0.794 last_best 5
epoch 15 loss_train 0.818 loss_eval 0.981 last_best 6
epoch 16 loss_train 0.763 loss_eva

## Avaliação

In [29]:
model.eval()
lres = []
ytrue = []
with torch.no_grad():
    for data,target in dl_test:
        data = data.to(device)
        pred = model(data)
        res  = pred.argmax(dim=1).cpu().tolist()
        lres += res
        ytrue += target

In [30]:
print(metrics.classification_report(ytrue,lres))

              precision    recall  f1-score   support

           0       0.74      0.47      0.57        30
           1       0.61      0.83      0.70        30

    accuracy                           0.65        60
   macro avg       0.67      0.65      0.64        60
weighted avg       0.67      0.65      0.64        60



Classification report - regnet_x_400mf
```
                precision    recall  f1-score   support

           0       0.74      0.47      0.57        30
           1       0.61      0.83      0.70        30

    accuracy                           0.65        60
   macro avg       0.67      0.65      0.64        60
weighted avg       0.67      0.65      0.64        60


```


Classification report - mobilenet_v2

```
precision    recall  f1-score   support

           0       0.63      0.63      0.63        30
           1       0.63      0.63      0.63        30

    accuracy                           0.63        60
   macro avg       0.63      0.63      0.63        60
weighted avg       0.63      0.63      0.63        60

```

In [44]:
img = image_loader("./cachorro/20241021_173505.jpg")
img = transform(img)

In [45]:
img = img.to(device)
img = img.unsqueeze(0)

In [46]:
model.to(device)

RegNet(
  (stem): SimpleStemIN(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (trunk_output): Sequential(
    (block1): AnyStage(
      (block1-0): ResBottleneckBlock(
        (proj): Conv2dNormActivation(
          (0): Conv2d(32, 48, kernel_size=(1, 1), stride=(2, 2), bias=False)
          (1): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (f): BottleneckTransform(
          (a): Conv2dNormActivation(
            (0): Conv2d(32, 48, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (1): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU(inplace=True)
          )
          (b): Conv2dNormActivation(
            (0): Conv2d(48, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=6, bias=False)
            

In [47]:
pred = model(img)

In [48]:
probabilities = F.softmax(pred, dim=1)

class_idx = torch.argmax(probabilities).item()

In [49]:
class_idx

0

# Bot

In [31]:
!pip install python-telegram-bot --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 652.1/652.1 kB 17.0 MB/s eta 0:00:00


In [32]:
from telegram import ForceReply, Update
from telegram.ext import Application, CommandHandler, ContextTypes, MessageHandler, filters, Updater


In [33]:
async def start(update: Update, context):
    await update.message.reply_text("Olá! Envie uma imagem e direi se é um cachorro ou um gato!")

In [43]:
async def echo(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    try:
      imagem = update.message.photo[-1]
      #await update.message.reply_text("Imagem recebida:", imagem.file_id)
    except Exception as e:
        await update.message.reply_text(f"Houve um erro: {str(e)}")
    file = await imagem.get_file()
    path = 'temp.jpg'
    await file.download_to_memory(path)
    print(f"Imagem salva em: {path}")
    img = image_loader(path)
    img = transform(img)
    img = img.unsqueeze(0)
    img = img.to(device)
    model.to(device)
    try:
      pred = model(img)
      print("Saída do modelo:", pred)

      probabilities = F.softmax(pred, dim=1)

      class_idx = torch.argmax(probabilities).item()
      msg = ''
      if class_idx == 0:
          msg = 'Cachorro'
      elif class_idx == 1:
          msg = 'Gato'
      else:
          msg = 'Não consegui identificar.'
      await update.message.reply_text(msg)
    except Exception as e:
      print("Erro durante a predição:", e)
      await update.message.reply_text("Houve um erro durante a predição.")

In [36]:
async def text_msg(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
   await update.message.reply_text("Envie uma imagem.")

In [39]:
async def main2() -> None:

    # Create the Application and pass it your bot's token.
    application = Application.builder().token("7948228837:AAGt2kRptvhHSMbogJbr8croCSekyd4YpqM").build()
    await application.initialize()

    # on non command i.e message - echo the message on Telegram
    application.add_handler(CommandHandler("start", start))
    application.add_handler(MessageHandler(filters.PHOTO, echo))
    application.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, text_msg))
    application.add_handler(CommandHandler("stop", stop))
    await application.start()
    await application.updater.start_polling()


await main2()
